In [43]:
from tqsdk import TqApi, TqSim, TqAuth, TqKq
import datetime
import time
tianqin = TqApi(TqKq(), auth=TqAuth('x6504368', 'q6504368'))

Task was destroyed but it is pending!
task: <Task pending name='Task-6' coro=<TqApi._notify_watcher() running at c:\Users\96541\anaconda3\envs\DeepLearning\Lib\site-packages\tqsdk\api.py:3793> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-5' coro=<TqBaseApi._windows_patch() running at c:\Users\96541\anaconda3\envs\DeepLearning\Lib\site-packages\tqsdk\baseApi.py:146> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[TqBaseApi._on_task_done()]>
Exception ignored in: <coroutine object TqApi._notify_watcher at 0x000002956B1D1540>
Traceback (most recent call last):
  File "c:\Users\96541\anaconda3\envs\DeepLearning\Lib\site-packages\tqsdk\api.py", line 3793, in _notify_watcher
    async for _ in update_chan:
  File "c:\Users\96541\anaconda3\envs\DeepLearning\Lib\site-packages\tqsdk\channel.py", line 143, in __anext__
    value = await asyncio.Queue.get(self)
            ^^^^^^^^^^^^^

2025-10-27 23:11:28 -     INFO - 通知 x6504368: 与 wss://otg-sim.shinnytech.com/trade 的网络连接已建立
2025-10-27 23:11:28 -     INFO - 通知 : 与 wss://free-api.shinnytech.com/t/nfmd/front/mobile 的网络连接已建立
2025-10-27 23:11:28 -     INFO - 通知 x6504368: 登录成功


In [44]:
def trans_time(ts):
    try:
        from datetime import datetime
        timestamp_s = ts / 1e9
        dt_object = datetime.fromtimestamp(timestamp_s)
        formatted_time = dt_object.strftime("%Y-%m-%d %H:%M:%S")
        return formatted_time
    except:
        return ts
data = tianqin.get_kline_serial('SHFE.au2512', duration_seconds=60, data_length=10000)
df = data.copy().sort_values("datetime")
df['datetime'] = df['datetime'].apply(trans_time)
# df = df[df['datetime']>=df[df['datetime'].str.contains(" 21:00")]['datetime'].iloc[0]]
tianqin.wait_update()
df

,datetime,id,open,high,low,close,volume,open_oi,close_oi,symbol,duration
0,2025-09-23 09:17:00,114659.0,855.20,855.40,855.02,855.02,1128.0,271078.0,271278.0,SHFE.au2512,60
1,2025-09-23 09:18:00,114660.0,855.02,855.04,854.66,854.98,1086.0,271278.0,271315.0,SHFE.au2512,60
2,2025-09-23 09:19:00,114661.0,854.98,855.04,854.70,854.88,838.0,271315.0,271408.0,SHFE.au2512,60
3,2025-09-23 09:20:00,114662.0,854.88,854.92,854.56,854.78,618.0,271408.0,271495.0,SHFE.au2512,60
4,2025-09-23 09:21:00,114663.0,854.78,854.80,854.12,854.12,864.0,271495.0,271473.0,SHFE.au2512,60
...,...,...,...,...,...,...,...,...,...,...,...
9995,2025-10-27 23:07:00,124654.0,913.60,913.64,912.66,913.16,1474.0,181155.0,181259.0,SHFE.au2512,60
9996,2025-10-27 23:08:00,124655.0,913.16,913.46,912.82,912.82,1223.0,181259.0,181434.0,SHFE.au2512,60
9997,2025-10-27 23:09:00,124656.0,912.82,912.92,912.30,912.76,1518.0,181434.0,181466.0,SHFE.au2512,60
9998,2025-10-27 23:10:00,124657.0,912.76,913.24,912.52,912.62,1175.0,181466.0,181535.0,SHFE.au2512,60
